In [2]:
import keras, os, pickle, re, sklearn, string, tensorflow
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.layers import Embedding
from keras.optimizers import Adadelta, adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils.vis_utils  import plot_model

print('Keras version: \t\t%s' % keras.__version__)
print('Scikit version: \t%s' % sklearn.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)


Using TensorFlow backend.


Keras version: 		2.0.9
Scikit version: 	0.19.1
TensorFlow version: 	1.4.1


In [5]:
import nltk
from nltk.corpus import stopwords
from tika import parser
from sklearn.metrics import classification_report



# Load Data

In [7]:
def LoadData_resume():
    label_resume=[]
    i=0
    for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level1/resum"):
        try:
            print (i, file)
            parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level1/resum/"+file)

            resume_contents=clean_doc(parsedPDF['content'])
            #Data = resume_contents.encode('utf-8')    
            label_resume.append((resume_contents,'Resume'))
        except UnicodeEncodeError:
            print ('Unicode error:', file)
        i=i+1
    #print (label_resume)
    return(label_resume)
        

In [8]:
def LoadData_JobDescription():
    label_jd=[]
    i=1
    for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level1/jd"):
        try:
            print (i, file)
            parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level1/jd/"+file)
        
            jd_contents=clean_doc(parsedPDF['content'])
       
            
            label_jd.append((jd_contents, 'JobDescription'))
        except UnicodeEncodeError:
            print ('Unicode error:', file)
        i=i+1
            
    return(label_jd)

# Preprocessing

In [9]:

def clean_doc(doc):
    """
    Cleaning a document by several methods:
        - Lowercase
        - Removing whitespaces
        - Removing stopwords
        - Removing punctuations
        
    """
    stop_words = set(stopwords.words('english'))
    
    # Lowercase
    doc = doc.lower()
   
    tokens = doc.translate({ord(c):"" for c in "\u200b\uf020\u2028\xa0\uf0e0\uf095\uf041\uf0e1\uf0b7\xad"})
    tokens = tokens.translate({ord(c):" " for c in "[):,·](;•●■♦▪"})
    tokens = tokens.translate({ord(c):"f" for c in "�"})
    
    # Removing multiple whitespaces
    tokens = re.sub(r"\?", " \? ", tokens)
    
    # Split in tokens
    tokens = tokens.split()
    
    # Remove Stopwords
    tokens = [w for w in tokens if not w in stop_words]
    # Remove punctuation
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    
    return ' '.join(tokens)

In [7]:
label_resume=LoadData_resume()

0 DEEPAKJOSE[2_4].pdf
1 52559549_Hyderabad___Secunderabad_4.00_yrs.docx
2 Akshay_Chaudhary_Pune_10.06_yrs.doc
3 Abdul Rafi (1).doc
4 ankita_tailor_India_5.00_yrs.docx
5 jithasusanshaji[3_1].docx
6 Aravind Syam Prakash.doc
7 AnithaPG[6_0].doc
8 AdarshH[2_6].pdf
9 ANISHR[3_6].pdf
10 Amanjeet_Singh_Delhi_2.00_yrs.doc
11 AntonyXavierJ[15_0].docx
12 arifca[5_1].pdf
13 AnithaRaghavendra[2_3].pdf
14 ANI VARGHESE.doc
15 Abhilash Rajan.doc
16 83604512_Chennai_12.00_yrs.doc
17 Anoop C_Resume.pdf
18 56705422_Delhi_5.06_yrs.pdf
19 Ajith.docx
20 66432618_India_6.03_yrs.docx
21 48314531_India_2.09_yrs.pdf
22 84017585_Pune_6.00_yrs.doc
23 ARUNMA[5_3].doc
24 Content Writer -Interested.docx
25 ChevvaPradeep[6_0].docx
26 Ankit_Kumar_Bengaluru___Bangalore_1.02_yrs.docx
27 51379543_Hyderabad_-_Secunderabad_4.06_yrs.docx
28 Abin Davis - 3.8 yrs-rejected.docx
29 CV_Vibin.docx
30 AKHIL VISWAM.docx
31 Anitha_P_Bengaluru___Bangalore_0.00_yrs.docx
32 Anishkumar_A.doc
33 AlexJose-4.8 yrs.doc
34 Anish_Paul_Cochin

249 Ravi_M_Namakkal_0.06_yrs.pdf
250 Anusha A.S.doc
251 AnkitMurdia[6_0].pdf
252 Akhil[2_2].docx
253 AnkurDeep[5_0].docx
254 Anirban_Bandyopadhyay_Bengaluru___Bangalore_5.01_yrs.docx
255 56550931_Hyderabad___Secunderabad_5.09_yrs.doc
256 Abhay_Shukla_Noida_5.10_yrs.docx
257 Amit_Shah_Singapore_12.11_yrs.doc
258 45614955_Pune_9.10_yrs.docx
259 AbdulLatheef[2_7].pdf
260 Balasundaramk[2_0].doc
261 KATARIRAVIVARMA[1_5].doc
262 ANOOP JOY.doc
263 81940699_Bengaluru___Bangalore_2.00_yrs.pdf
264 ARUNM[1_9].doc
265 Amruth_Experience_CV.pdf
266 AneeshRKurup[7_0].doc
267 BibinPJacob.pdf
268 Amol_Ambhure_Pune_2.00_yrs.pdf
269 AnoopPB[4_8].docx
270 58055758_Bengaluru___Bangalore_5.02_yrs.docx
271 ArunKJacob[1_8].pdf
272 Aravind ps latest resume _17-Jun-16_11-18-46.pdf
273 25029877_Bengaluru___Bangalore_9.00_yrs.doc
274 chaithanya_thakellapati_Hyderabad___Secunderabad_5.10_yrs.docx
275 Rajnikant_bajpai___Nagpur_4.00_yrs.doc
276 JISHNUSSALIL[3_0].docx
277 Aabid_Khan_Ahmedabad_2.00_yrs.pdf
278 Ashutos

481 anilvishnu1(sh)15thJan'16.docx
482 Anu_Sathyanathan_CV- Reject.doc
483 Abhijeet_Singh_Bengaluru___Bangalore_5.10_yrs.doc
484 ANAND I MATHAD.doc
485 51045441_Bengaluru___Bangalore_3.05_yrs.doc
486 dineshkumar[8_0].pdf
487 Basil.K.pdf
488 ANSARSHERIEF[5_8].pdf
489 Balaji_D_Hyderabad___Secunderabad_8.02_yrs.docx
490 Arun[4_11].docx
491 Joice P Joy- Reject.doc
492 Ajay_ NET Developer.docx
493 Aalhad_Vengurlekar_Mumbai_4.02_yrs.docx
494 AMLAN_Das_Canada_6.10_yrs.doc
495 A.R.Umesh.docx
496 ARATHY  PRASANNAN.pdf
497 A.Mohana Krishnan_Atlas Systems.doc
498 56116215_Hyderabad_-_Secunderabad_5.05_yrs.docx
499 Adhish Bhoi.doc


In [8]:
len(label_resume)

500

In [9]:
label_jd=LoadData_JobDescription()

1 Senior Software Engineer - PHP
2 Marketing Executive Tele (Hindi)
3 SEO Analyst
4 infopark_jd_1713
5 Training Coordinator - Part time
6 infopark_jd_231
7 infopark_jd_282
8 infopark_jd_1174
9 infopark_jd_80
10 Business Development Associate
11 infopark_jd_160
12 Senior PHP Developer
13 infopark_jd_806
14 infopark_jd_1153
15 infopark_jd_146
16 MSSQL Database Administrator
17 infopark_jd_97
18 infopark_jd_22
19 infopark_jd_799
20 PMO Executive
21 infopark_jd_89
22 Software_Sr.Software Engineer (RoR)
23 infopark_jd_113
24 infopark_jd_785
25 UX _ UI Designer
26 infopark_jd_86
27 infopark_jd_96
28 Software Engineer (.NET)
29 infopark_jd_1662
30 Java_Angular JS Developers
31 infopark_jd_195
32 UI_ UX Designer - Web_ Mobile Apps
33 infopark_jd_218
34 infopark_jd_331
35 infopark_jd_31
36 infopark_jd_793
37 Corejava Developer
38 infopark_jd_616
39 Junior UI Developer
40 Digital marketing trainee
41 Business Intelligence Specialist
42 Associate Software Engineer-Trainee
43 infopark_jd_73
44 inf

336 infopark_jd_628
337 infopark_jd_1718
338 Engineer - Cloud
339 infopark_jd_463
340 infopark_jd_1585
341 infopark_jd_104
342 infopark_jd_302
343 infopark_jd_284
344 infopark_jd_115
345 infopark_jd_774
346 infopark_jd_283
347 Software Engineer
348 PHP Developers
349 infopark_jd_196
350 infopark_jd_1720
351 Senior Quality Controller
352 infopark_jd_345
353 Junior Presales Executive
354 infopark_jd_239
355 Business Development Executive
356 infopark_jd_159
357 infopark_jd_1158
358 infopark_jd_747
359 infopark_jd_216
360 infopark_jd_579
361 infopark_jd_790
362 IOS  Developer
363 Walkin - Automation Test Engineer
364 Marketing Associate
365 Node.js Developer - Javascript_coffeescript
366 Office Administrator
367 infopark_jd_94
368 infopark_jd_776
369 Android Developers
370 Senior Java Developer
371 Content programming and aggregation trainee
372 infopark_jd_334
373 infopark_jd_281
374 infopark_jd_81
375 Journalist  - Sports ( English Writing Skills )
376 Senior Android Developer(Location-

In [10]:

print(len(label_resume))
print(len(label_jd))

500
500


In [11]:

jdres_train_labels=[]
jdres_train_data=[]

for row in label_resume:
    jdres_train_data.append(row[0])
    jdres_train_labels.append(row[1])


In [13]:

for row in label_jd:
    jdres_train_data.append(row[0])
    jdres_train_labels.append(row[1])
#print(jdres_train_labels)
#print(jdres_train_data)

In [14]:
print(len(jdres_train_data))
print(len(jdres_train_labels))

1000
1000


In [15]:
#print(jdres_train_data[42])

In [16]:
labels = [0 for _ in range(len(label_resume))] + [1 for _ in range(len(label_jd))]
#print(labels)
print(len(labels))

1000


In [17]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(jdres_train_data)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(jdres_train_data)
vocab_size

27280

In [18]:
# pad documents to a max length of 50 words
max_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[  129   564  2514 ...,   503  1468   548]
 [  611   352   980 ...,  2182  1760   768]
 [  174   511    51 ...,  8721  8722  8723]
 ..., 
 [  609    10   123 ...,    10   699 12340]
 [  172    46    81 ...,   394  1760  7675]
 [  951   644   128 ...,     0     0     0]]


In [19]:
labels = to_categorical(labels,num_classes=2)
print(len(labels))
labels

1000


array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

# Loading Glove

In [20]:
#loading glove

embeddings_index = dict()
f = open('/home/shabna/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [21]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Create Model

In [22]:
#create model

from keras.layers import Activation, Input, Dense, Flatten, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D,GlobalMaxPooling1D
from keras.models import Model

In [23]:
#from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [410]:
def create_model():
    model = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=50, trainable='true')
    model.add(e)
    model.add(Conv1D(100, 5, activation='relu',name='l1'))
    model.add(MaxPooling1D(pool_size=5,name='l2'))
    model.add(Conv1D(100, 5, activation='relu',name='l3'))
    model.add(GlobalMaxPooling1D(name='l4'))
    #model.add(Dense(100, activation='relu',name='l5'))
    model.add(Dropout(0.2))
    #model.add(Flatten())
    model.add(Dense(2, activation='softmax',name='l6'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    return model

In [411]:

model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 50, 100)           2728000   
_________________________________________________________________
l1 (Conv1D)                  (None, 46, 100)           50100     
_________________________________________________________________
l2 (MaxPooling1D)            (None, 9, 100)            0         
_________________________________________________________________
l3 (Conv1D)                  (None, 5, 100)            50100     
_________________________________________________________________
l4 (GlobalMaxPooling1D)      (None, 100)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)               0         
_________________________________________________________________
l6 (Dense)                   (None, 2)                 202       
Total para

In [412]:
from keras.callbacks import ModelCheckpoint

In [413]:
from keras.callbacks import Callback

In [414]:
filepath="/home/shabna/Desktop/example_codes/weights_1.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only="True", mode="max")
callbacks_list = [checkpoint]

# Cross Validation

In [415]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)

XX_train, X_test, yy_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=42)

i=1
for train_index, test_index in kf.split(XX_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    print(i,":num of fold")

    X_train, X_val = XX_train[train_index], XX_train[test_index]
    y_train, y_val = yy_train[train_index], yy_train[test_index]
    
    history = model.fit(X_train, y_train, validation_data=(X_val,y_val) ,epochs=50, batch_size=100, verbose=1, callbacks=callbacks_list)
   
    i=i+1
    

1 :num of fold
Train on 533 samples, validate on 267 samples
Epoch 1/50
533/533 [==============================] - 5s 9ms/step - loss: 0.7782 - acc: 0.5985 - val_loss: 0.5779 - val_acc: 0.7079
Epoch 2/50
533/533 [==============================] - 2s 3ms/step - loss: 0.4705 - acc: 0.7749 - val_loss: 0.3756 - val_acc: 0.8315
Epoch 3/50
533/533 [==============================] - 1s 2ms/step - loss: 0.2924 - acc: 0.8762 - val_loss: 0.3081 - val_acc: 0.8727
Epoch 4/50
533/533 [==============================] - 1s 2ms/step - loss: 0.2173 - acc: 0.9325 - val_loss: 0.2532 - val_acc: 0.9064
Epoch 5/50
533/533 [==============================] - 1s 2ms/step - loss: 0.1593 - acc: 0.9456 - val_loss: 0.2442 - val_acc: 0.8989
Epoch 6/50
533/533 [==============================] - 1s 2ms/step - loss: 0.1114 - acc: 0.9700 - val_loss: 0.2005 - val_acc: 0.9176
Epoch 7/50
533/533 [==============================] - 1s 2ms/step - loss: 0.0854 - acc: 0.9812 - val_loss: 0.2064 - val_acc: 0.9139
Epoch 8/50
533/

Epoch 31/50
533/533 [==============================] - 1s 2ms/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.1563 - val_acc: 0.9288
Epoch 32/50
533/533 [==============================] - 2s 3ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.1507 - val_acc: 0.9363
Epoch 33/50
533/533 [==============================] - 1s 2ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.1663 - val_acc: 0.9288
Epoch 34/50
533/533 [==============================] - 1s 2ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.1629 - val_acc: 0.9288
Epoch 35/50
533/533 [==============================] - 1s 2ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.1574 - val_acc: 0.9326
Epoch 36/50
533/533 [==============================] - 1s 2ms/step - loss: 9.0839e-04 - acc: 1.0000 - val_loss: 0.1608 - val_acc: 0.9326
Epoch 37/50
533/533 [==============================] - 1s 2ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.1664 - val_acc: 0.9288
Epoch 38/50
533/533 [==============================] - 1s 2ms/ste

Epoch 12/50
533/533 [==============================] - 1s 2ms/step - loss: 4.3173e-04 - acc: 1.0000 - val_loss: 6.8744e-04 - val_acc: 1.0000
Epoch 13/50
533/533 [==============================] - 1s 2ms/step - loss: 3.8344e-04 - acc: 1.0000 - val_loss: 6.9745e-04 - val_acc: 1.0000
Epoch 14/50
533/533 [==============================] - 1s 2ms/step - loss: 2.7962e-04 - acc: 1.0000 - val_loss: 6.8023e-04 - val_acc: 1.0000
Epoch 15/50
533/533 [==============================] - 1s 2ms/step - loss: 3.2528e-04 - acc: 1.0000 - val_loss: 6.3841e-04 - val_acc: 1.0000
Epoch 16/50
533/533 [==============================] - 1s 2ms/step - loss: 2.4874e-04 - acc: 1.0000 - val_loss: 6.0334e-04 - val_acc: 1.0000
Epoch 17/50
533/533 [==============================] - 1s 2ms/step - loss: 2.8680e-04 - acc: 1.0000 - val_loss: 5.7284e-04 - val_acc: 1.0000
Epoch 18/50
533/533 [==============================] - 1s 2ms/step - loss: 2.5361e-04 - acc: 1.0000 - val_loss: 5.5620e-04 - val_acc: 1.0000
Epoch 19/50
5

Epoch 44/50
533/533 [==============================] - 1s 2ms/step - loss: 1.5388e-04 - acc: 1.0000 - val_loss: 4.5198e-04 - val_acc: 1.0000
Epoch 45/50
533/533 [==============================] - 1s 2ms/step - loss: 1.1316e-04 - acc: 1.0000 - val_loss: 4.9946e-04 - val_acc: 1.0000
Epoch 46/50
533/533 [==============================] - 1s 2ms/step - loss: 1.2384e-04 - acc: 1.0000 - val_loss: 5.1312e-04 - val_acc: 1.0000
Epoch 47/50
533/533 [==============================] - 1s 2ms/step - loss: 1.1174e-04 - acc: 1.0000 - val_loss: 4.9649e-04 - val_acc: 1.0000
Epoch 48/50
533/533 [==============================] - 1s 2ms/step - loss: 9.3684e-05 - acc: 1.0000 - val_loss: 4.8664e-04 - val_acc: 1.0000
Epoch 49/50
533/533 [==============================] - 1s 2ms/step - loss: 1.1674e-04 - acc: 1.0000 - val_loss: 4.8680e-04 - val_acc: 1.0000
Epoch 50/50
533/533 [==============================] - 1s 2ms/step - loss: 1.1215e-04 - acc: 1.0000 - val_loss: 5.0735e-04 - val_acc: 1.0000
3 :num of fol

# Load Model

In [3]:
model=load_model("weights_1.best.hdf5")


In [6]:
plot_model(model, to_file = 'models.png')

In [6]:
#y_train

In [418]:
from sklearn.metrics import classification_report, confusion_matrix

In [419]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred,axis=1)

In [420]:
y_pred = model.predict_classes(X_test)

200/200 [==============================] - 0s 573us/step


In [421]:
p = model.predict_proba(X_test)

200/200 [==============================] - 0s 621us/step


In [422]:
model.evaluate(X_test,y_test)

200/200 [==============================] - 1s 3ms/step


[0.092443118393421175, 0.96999999999999997]

In [423]:
model.evaluate(X_train,y_train)

534/534 [==============================] - 0s 550us/step


[0.018659200046031842, 0.99625468164794007]

In [424]:
labels_index = {'Resume':0 ,'JobDescription':1}
target_name = [t for t in labels_index.keys()]

In [425]:
print(classification_report(np.argmax(y_test,axis=1),y_pred,target_names=target_name))

                precision    recall  f1-score   support

        Resume       0.95      0.99      0.97        96
JobDescription       0.99      0.95      0.97       104

   avg / total       0.97      0.97      0.97       200



In [ ]:
print(confusion_matrix(np.argmax(y_test,axis=1),y_pred))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Prediction


In [17]:

parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level1/jd/Corejava Developer")

contents=[clean_doc(parsedPDF['content'])]
        
print(contents)

['corejava developer closing 31032017 contact email hrtvmarssoftwarecom brief description 2to 5years experience core java development candidate good knowledge developing windows applications sound fundamentals operating systems processes threads compilers tcpip networks must haves']


In [18]:
def prpInput(pp):
    tx = Tokenizer()
    tx.fit_on_texts(pp)
    vocab_size = len(tx.word_index)+1
    encoded_docs = tx.texts_to_sequences(pp)
    print(encoded_docs, len(encoded_docs))
    max_length = 50
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    return padded_docs

In [19]:

mat=prpInput(contents)
mat

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]] 1


array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [20]:
prd = model.predict(mat)
print (prd)

[[ 0.18099865  0.81900132]]


In [21]:
labels_index = {'Resume':0 ,'JobDescription':1}
rev_lable_index = {}
for key in labels_index:
    rev_lable_index[labels_index[key]] = key
print(rev_lable_index)
def result(prd,contents):
    y_classes = prd.argmax(axis=-1)
    print("num of prediction:",len( y_classes))
    lx=[]
    for idx,lb in enumerate(y_classes):
        lx.append([contents[idx],rev_lable_index[lb]])
    return lx

{0: 'Resume', 1: 'JobDescription'}


In [22]:
result(prd,contents)

num of prediction: 1


[['corejava developer closing 31032017 contact email hrtvmarssoftwarecom brief description 2to 5years experience core java development candidate good knowledge developing windows applications sound fundamentals operating systems processes threads compilers tcpip networks must haves',
  'JobDescription']]

In [23]:
import pandas as pd
pd.DataFrame(result(prd,contents),columns=['File Content','Label'])

num of prediction: 1


,File Content,Label
0,corejava developer closing 31032017 contact em...,JobDescription


# predictions from whole data

In [7]:
def test_load():
    i=0
    test=[]
    for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level1/resum"):
        print (i, file)
        parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level1/resum/"+file)
        content = clean_doc(parsedPDF['content'])
        test.append(content)
        #mat=prpInput(contents)
        #prd = model.predict(mat)
        #print(result(prd,contents))
        #pd.DataFrame(result(prd,contents),columns=['File Content','Label'])
        i=i+1
        
    return test
testinput = test_load()

0 DEEPAKJOSE[2_4].pdf
1 52559549_Hyderabad___Secunderabad_4.00_yrs.docx
2 Akshay_Chaudhary_Pune_10.06_yrs.doc
3 Abdul Rafi (1).doc
4 ankita_tailor_India_5.00_yrs.docx
5 jithasusanshaji[3_1].docx
6 Aravind Syam Prakash.doc
7 AnithaPG[6_0].doc
8 AdarshH[2_6].pdf
9 ANISHR[3_6].pdf
10 Amanjeet_Singh_Delhi_2.00_yrs.doc
11 AntonyXavierJ[15_0].docx
12 arifca[5_1].pdf
13 AnithaRaghavendra[2_3].pdf
14 ANI VARGHESE.doc
15 Abhilash Rajan.doc
16 83604512_Chennai_12.00_yrs.doc
17 Anoop C_Resume.pdf
18 56705422_Delhi_5.06_yrs.pdf
19 Ajith.docx
20 66432618_India_6.03_yrs.docx
21 48314531_India_2.09_yrs.pdf
22 84017585_Pune_6.00_yrs.doc
23 ARUNMA[5_3].doc
24 Content Writer -Interested.docx
25 ChevvaPradeep[6_0].docx
26 Ankit_Kumar_Bengaluru___Bangalore_1.02_yrs.docx
27 51379543_Hyderabad_-_Secunderabad_4.06_yrs.docx
28 Abin Davis - 3.8 yrs-rejected.docx
29 CV_Vibin.docx
30 AKHIL VISWAM.docx
31 Anitha_P_Bengaluru___Bangalore_0.00_yrs.docx
32 Anishkumar_A.doc
33 AlexJose-4.8 yrs.doc
34 Anish_Paul_Cochin

249 Ravi_M_Namakkal_0.06_yrs.pdf
250 Anusha A.S.doc
251 AnkitMurdia[6_0].pdf
252 Akhil[2_2].docx
253 AnkurDeep[5_0].docx
254 Anirban_Bandyopadhyay_Bengaluru___Bangalore_5.01_yrs.docx
255 56550931_Hyderabad___Secunderabad_5.09_yrs.doc
256 Abhay_Shukla_Noida_5.10_yrs.docx
257 Amit_Shah_Singapore_12.11_yrs.doc
258 45614955_Pune_9.10_yrs.docx
259 AbdulLatheef[2_7].pdf
260 Balasundaramk[2_0].doc
261 KATARIRAVIVARMA[1_5].doc
262 ANOOP JOY.doc
263 81940699_Bengaluru___Bangalore_2.00_yrs.pdf
264 ARUNM[1_9].doc
265 Amruth_Experience_CV.pdf
266 AneeshRKurup[7_0].doc
267 BibinPJacob.pdf
268 Amol_Ambhure_Pune_2.00_yrs.pdf
269 AnoopPB[4_8].docx
270 58055758_Bengaluru___Bangalore_5.02_yrs.docx
271 ArunKJacob[1_8].pdf
272 Aravind ps latest resume _17-Jun-16_11-18-46.pdf
273 25029877_Bengaluru___Bangalore_9.00_yrs.doc
274 chaithanya_thakellapati_Hyderabad___Secunderabad_5.10_yrs.docx
275 Rajnikant_bajpai___Nagpur_4.00_yrs.doc
276 JISHNUSSALIL[3_0].docx
277 Aabid_Khan_Ahmedabad_2.00_yrs.pdf
278 Ashutos

481 anilvishnu1(sh)15thJan'16.docx
482 Anu_Sathyanathan_CV- Reject.doc
483 Abhijeet_Singh_Bengaluru___Bangalore_5.10_yrs.doc
484 ANAND I MATHAD.doc
485 51045441_Bengaluru___Bangalore_3.05_yrs.doc
486 dineshkumar[8_0].pdf
487 Basil.K.pdf
488 ANSARSHERIEF[5_8].pdf
489 Balaji_D_Hyderabad___Secunderabad_8.02_yrs.docx
490 Arun[4_11].docx
491 Joice P Joy- Reject.doc
492 Ajay_ NET Developer.docx
493 Aalhad_Vengurlekar_Mumbai_4.02_yrs.docx
494 AMLAN_Das_Canada_6.10_yrs.doc
495 A.R.Umesh.docx
496 ARATHY  PRASANNAN.pdf
497 A.Mohana Krishnan_Atlas Systems.doc
498 56116215_Hyderabad_-_Secunderabad_5.05_yrs.docx
499 Adhish Bhoi.doc


In [36]:
h=prpInput(testinput)
h

[[3732, 3233, 3732, 3233, 487, 12176, 12177, 1374, 1165, 900, 12178, 197, 9, 8321, 12179, 292, 232, 12180, 12181, 57, 12182, 150, 126, 753, 309, 150, 1929, 72, 290, 665, 12183, 24, 15, 764, 419, 57, 61, 161, 155, 1307, 57, 61, 175, 367, 178, 175, 367, 61, 153, 190, 178, 153, 190, 61, 631, 5, 1603, 118, 303, 118, 1013, 54, 2, 2147, 1029, 2492, 565, 334, 505, 342, 99, 57, 61, 1604, 161, 471, 1341, 244, 5623, 8322, 213, 57, 12184, 376, 244, 4430, 57, 631, 982, 235, 57, 287, 217, 12185, 274, 649, 2405, 17, 144, 482, 203, 11, 534, 244, 8323, 4986, 57, 116, 998, 1099, 57, 672, 26, 25, 244, 2615, 6661, 12186, 11, 8324, 631, 535, 1813, 2233, 48, 631, 81, 1375, 244, 2615, 63, 140, 1554, 687, 631, 382, 3472, 12187, 3472, 808, 3473, 970, 3473, 3472, 5624, 3472, 1342, 1643, 3472, 50, 244, 1277, 808, 3473, 970, 3473, 5625, 1698, 57, 61, 43, 213, 408, 3234, 1698, 542, 17, 244, 4431, 11, 63, 57, 376, 2234, 437, 244, 316, 61, 1376, 12188, 2882, 1308, 3235, 1309, 244, 316, 12189, 57, 1225, 244, 316, 13

array([[12202,  4434,   130, ...,   512,  1644,   500],
       [ 1460,   218,   346, ...,  3244,  3059,   716],
       [  767,   239,   186, ...,  6683,  6682,   295],
       ..., 
       [   34, 27848,   461, ..., 27858,    46, 27859],
       [  220,   399,   106, ...,   543,   239,   186],
       [    3,  1753,   155, ...,   177,   197,  8319]], dtype=int32)

In [37]:
prdi = model.predict(h)

InvalidArgumentError: indices[31,4] = 27284 is not in [0, 27280)
	 [[Node: embedding_8_2/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_8_2/embeddings/read, embedding_8_2/Cast)]]

Caused by op 'embedding_8_2/Gather', defined at:
  File "/home/shabna/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/shabna/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/shabna/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/shabna/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/shabna/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/shabna/anaconda3/lib/python3.

In [ ]:
rev_lable_index = {}
for key in labels_index:
    rev_lable_index[labels_index[key]] = key
print(rev_lable_index)


def result(prdi, testinput):
    #prdi = model.predict(h)
    y_classes = prdi.argmax(axis=-1)
    print("num of target items:",len(y_classes))
    lx=[]
    for idx,lb in enumerate(y_classes):
        lx.append((testinput[idx],rev_lable_index[lb]))
    return lx

In [ ]:
result(prdi,testinput)

In [ ]:
lx_label=[]
for row in lx:
    lx_label.append(row[1])
c=0
for i in lx_label:
    if i=="Resume":
        #print (i)
        c=c+1
print(c)

In [ ]:
import pandas as pd
pd.DataFrame(result(prdi,testinput),columns=['File Content','Label'])